In [52]:
import os
import numpy as np

#from sklearn.metrics.pairwise import cosine_similarity
from numpy import dot
from numpy.linalg import norm

In [53]:
# Load Glove Embedding
gloveFile = "glove.6B/glove.6B.300d.txt"
glove_model = {}

with open(gloveFile, 'r') as f:
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        glove_model[word] = embedding
    print "Done.",len(glove_model)," words loaded!"  

print glove_model['unk']

Done. 400000  words loaded!
[  3.00710000e-01  -4.68670000e-01  -2.06170000e-01  -8.09780000e-01
  -2.38890000e-01   2.43290000e-01   1.65380000e-02  -3.56870000e-02
  -2.23060000e-01   9.51890000e-01  -3.22730000e-01   2.19800000e-01
  -6.75240000e-02  -3.72200000e-01  -3.97180000e-01  -4.38610000e-01
   1.19670000e-01  -2.99640000e-01   2.84370000e-02  -8.75440000e-02
   1.65690000e-01  -4.94510000e-01  -6.20110000e-01  -1.65740000e-01
  -9.72180000e-02  -9.94740000e-02  -8.03070000e-02  -3.93380000e-01
  -2.41950000e-01   3.20230000e-01  -5.33200000e-01  -4.01840000e-01
  -6.71350000e-01  -7.85610000e-02   5.55460000e-01   2.99970000e-01
  -9.96500000e-02  -6.70350000e-01   1.26690000e-01  -1.86180000e-01
  -6.26210000e-02   4.52900000e-01   3.92650000e-01   2.41210000e-01
  -4.14740000e-01  -6.18900000e-01  -1.04120000e-01  -3.10430000e-01
  -6.67880000e-03  -8.32480000e-01   6.51500000e-01   9.01810000e-01
   2.41460000e-02  -7.07660000e-02  -3.95800000e-01  -3.64870000e-01
  -2.3

In [59]:
# Prepare dataset for NLTK HMM Tagger
# Divide in train and test files [80:20] 

# Directory having content
doc_dir = 'Data/content'

train_file_list = []
test_file_list = []

for f in os.listdir(doc_dir):
    #Random Sampling
    if np.random.uniform(0,1)< 0.8:
        train_file_list.append(f)
    else:
        test_file_list.append(f)

print len(train_file_list)
print len(test_file_list)

1024
283


In [68]:
# Directory having tags
tag_dir = 'Data/new_tags'

tagEmbedding = dict()
tagCount = dict()
win_size = 0
word_dim = 300

for f in train_file_list:
    print 'Processing file '+f
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    lines_in_word_file = []
    lines_in_tag_file = []
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.add(word_embed, glove_model[word_in_file[j-win_size+k].lower()])
                        else:
                            word_embed = np.add(word_embed, glove_model['unk'])  
                    # Create embdedding of Tag    
                    if tag_in_file[j] in tagEmbedding:
                        tagEmbedding[tag_in_file[j]] = np.add(tagEmbedding[tag_in_file[j]], word_embed)
                        tagCount[tag_in_file[j]] = tagCount[tag_in_file[j]] + 1
                    else:
                        tagEmbedding[tag_in_file[j]] = word_embed
                        tagCount[tag_in_file[j]] = 1
                    

Processing file 2010_12_7_st-59.txt
Processing file ev_081_st_014.txt
Processing file ev_079_st_013.txt
Processing file ev_089_st_033.txt
Processing file ev_073_st_012.txt
Processing file 2013_6_17_st-719.txt
Processing file 2012_6_11_st-43.txt
Processing file ev_037_st_011.txt
Processing file ev_075_st_004.txt
Processing file 2011_1_9_st-454.txt
Processing file 2012_9_25_st-79.txt
Processing file 2013_4_23_st-470.txt
Processing file ev_050_st_006.txt
Processing file 2013_5_18_st-88.txt
Processing file ev_064_st_014.txt
Processing file 2011_7_24_st-5.txt
Processing file ev_090_st_012.txt
Processing file ev_039_st_010.txt
Processing file ev_051_st_007.txt
Processing file ev_029_st_001.txt
Processing file ev_079_st_005.txt
Processing file 2013_8_29_st-580.txt
Processing file ev_036_st_001.txt
Processing file 2014_11_11_st-4005.txt
Processing file ev_060_st_001.txt
Processing file 2012_8_2_st-724.txt
Processing file ev_050_st_002.txt
Processing file ev_039_st_018.txt
Processing file ev_07

Processing file 2012_10_6_st-100.txt
Processing file ev_039_st_019.txt
Processing file ev_064_st_009.txt
Processing file ev_086_st_011.txt
Processing file 2013_5_13_st-33.txt
Processing file ev_065_st_011.txt
Processing file ev_060_st_002.txt
Processing file ev_065_st_006.txt
Processing file 2011_10_29_st-723.txt
Processing file ev_086_st_033.txt
Processing file 2013_9_16_st-23.txt
Processing file ev_073_st_009.txt
Processing file 2013_7_10_st-69.txt
Processing file ev_025_st_001.txt
Processing file ev_065_st_003.txt
Processing file ev_094_st_025.txt
Processing file 2013_9_27_st-689.txt
Processing file ev_074_st_014.txt
Processing file 2011_2_2_st-18.txt
Processing file 2013_3_4_st-17.txt
Processing file ev_072_st_013.txt
Processing file ev_077_st_022.txt
Processing file ev_086_st_034.txt
Processing file 2013_6_10_st-287.txt
Processing file ev_074_st_013.txt
Processing file 2014_10_6_st-4004.txt
Processing file 2010_4_6_st-398.txt
Processing file 2010_7_14_st-41.txt
Processing file ev_

Processing file ev_088_st_001.txt
Processing file ev_084_st_025.txt
Processing file 2012_12_30_st-417.txt
Processing file ev_019_st_049.txt
Processing file ev_064_st_026.txt
Processing file 2011_1_30_st-709.txt
Processing file ev_035_st_014.txt
Processing file 2010_9_9_st-2.txt
Processing file 2013_2_23_st-463.txt
Processing file 2012_8_31_st-671.txt
Processing file 2013_7_4_st-4.txt
Processing file 2011_5_8_st-49.txt
Processing file ev_093_st_009.txt
Processing file 2011_6_21_st-55.txt
Processing file ev_089_st_029.txt
Processing file ev_081_st_010.txt
Processing file ev_068_st_004.txt
Processing file ev_066_st_005.txt
Processing file 2011_7_28_st-255.txt
Processing file 2012_4_30_st-166.txt
Processing file 2012_11_22_st-671.txt
Processing file ev_082_st_003.txt
Processing file 2010_8_21_st-384.txt
Processing file 2013_6_16_st-42.txt
Processing file 2013_9_25_st-108.txt
Processing file ev_088_st_013.txt
Processing file 2010_4_9_st-26.txt
Processing file 2010_7_27_st-43.txt
Processing 

Processing file 2013_2_21_st-121.txt
Processing file ev_071_st_002.txt
Processing file ev_056_st_001.txt
Processing file 2010_12_10_st-64.txt
Processing file ev_066_st_021.txt
Processing file 2013_4_19_st-42.txt
Processing file ev_014_st_006.txt
Processing file ev_092_st_024.txt
Processing file 2013_8_30_st-5.txt
Processing file 2013_3_8_st-478.txt
Processing file 2013_4_23_st-14.txt
Processing file 2013_7_11_st-59.txt
Processing file ev_064_st_007.txt
Processing file 2011_7_19_st-65.txt
Processing file 2011_9_9_st-44.txt
Processing file ev_088_st_003.txt
Processing file 2012_8_3_st-684.txt
Processing file 2010_11_27_st-23.txt
Processing file ev_077_st_008.txt
Processing file ev_064_st_027.txt
Processing file ev_076_st_010.txt
Processing file 2013_3_4_st-430.txt
Processing file 2013_5_12_st-65.txt
Processing file 2013_4_27_st-36.txt
Processing file ev_092_st_004.txt
Processing file ev_015_st_003.txt
Processing file 2011_9_8_st-4.txt
Processing file 2012_5_16_st-99.txt
Processing file 2

In [69]:
for key in tagCount:
    tagEmbedding[key] = tagEmbedding[key]/tagCount[key]
    
print tagCount

print '\nPER_Assoc\n'
print tagEmbedding['PER_Assoc']
print '\nORG_Assoc\n'
print tagEmbedding['ORG_Assoc']
print '\n'
print tagEmbedding['PER_Others']
print '\n'
print tagEmbedding['Victim']
print '\n'
print tagEmbedding['LOC_Event']
print '\n'
print tagEmbedding['O']
print '\n'
print tagEmbedding['ORG_Accused']
print '\n'
print tagEmbedding['PER_Victim']
print '\n'
print tagEmbedding['ORG_Victim']
print '\n'
print tagEmbedding['LOC_Others']
print '\n'
print tagEmbedding['ORG_Others']
print '\n'
print tagEmbedding['PER_Accused']
print '\n'
print tagEmbedding['LOC_Accused']
print '\n'
print tagEmbedding['LOC_Assoc']
print '\n'
print tagEmbedding['LOC_Victim']
print '\n'
print tagEmbedding['Event']
print '\n'
print tagEmbedding['Location']
                    

{'PER_Assoc': 225, 'ORG_Assoc': 60, 'PER_Others': 6887, 'Victim': 30, 'LOC_Event': 4983, 'Accused': 255, 'LOC_Others': 6864, 'ORG_Accused': 2223, 'PER_Victim': 1030, 'ORG_Victim': 314, 'O': 332877, 'ORG_Others': 8776, 'PER_Accused': 3302, 'LOC_Accused': 667, 'LOC_Assoc': 304, 'LOC_Victim': 186, 'Event': 302, 'Location': 516}

PER_Assoc

[ -8.41583876e-02   3.01661658e-03   2.13148850e-02  -1.26526303e-01
  -3.78960244e-02  -2.00014342e-02  -3.33154524e-02   1.04340239e-01
  -4.47704742e-02  -9.98278938e-01   9.15966436e-02   5.37587924e-02
  -5.80334982e-02   6.77263973e-02   5.25981422e-02   1.99412310e-01
  -8.96041919e-02  -6.40260578e-03  -3.41745440e-02  -5.42217378e-02
  -1.10219670e-02   6.81454133e-03   8.39141667e-02  -2.07552716e-03
  -1.78047618e-01  -8.90522342e-02   3.15476264e-02  -5.66638796e-02
  -1.03298301e-01  -2.81441298e-02   3.02924527e-02   1.49311988e-01
  -4.27908969e-02   1.23051636e-01  -3.30737511e-01  -1.19526686e-01
   7.11337524e-02   6.16264733e-02  -7.5

In [70]:
# Test
actual_tag_count = dict()
matched_tag_count = dict()
matched_tag_count['PER_Others'] = 0
actual_tag_count['PER_Others'] = 0
matched_tag_count['PER_Victim'] = 0
actual_tag_count['PER_Victim'] = 0
matched_tag_count['PER_Accused'] = 0
actual_tag_count['PER_Accused'] = 0
matched_tag_count['ORG_Victim'] = 0
actual_tag_count['ORG_Victim'] = 0
matched_tag_count['ORG_Accused'] = 0
actual_tag_count['ORG_Accused'] = 0
matched_tag_count['ORG_Others'] = 0
actual_tag_count['ORG_Others'] = 0
matched_tag_count['LOC_Accused'] = 0
actual_tag_count['LOC_Accused'] = 0
matched_tag_count['LOC_Others'] = 0
actual_tag_count['LOC_Others'] = 0
matched_tag_count['LOC_Event'] = 0
actual_tag_count['LOC_Event'] = 0
matched_tag_count['LOC_Victim'] = 0
actual_tag_count['LOC_Victim'] = 0

for f in test_file_list:
    word_file_path = os.path.join(doc_dir, f)
    tag_file_path = os.path.join(tag_dir, f)
    
    lines_in_word_file = []
    lines_in_tag_file = []
    
    with open(word_file_path, "rt") as word_file:
        for line in word_file:
            lines_in_word_file.append(line)
    with open(tag_file_path, "rt") as tag_file:
        for line in tag_file:
            lines_in_tag_file.append(line)
    if (len(lines_in_word_file) == len(lines_in_tag_file)) and len(lines_in_word_file) > 0:
        for i in xrange(len(lines_in_word_file)):
            word_in_file = lines_in_word_file[i].split()
            tag_in_file = lines_in_tag_file[i].split()
            length = min(len(word_in_file), len(tag_in_file))
            for j in xrange(length):
                # Get the word embedding of the word
                word_embed = np.zeros(word_dim)
                if (j>(win_size-1)) and (j<length-win_size):
                    for k in xrange(2*win_size + 1):
                        if word_in_file[j-win_size+k].lower() in glove_model:
                            word_embed = np.add(word_embed, glove_model[word_in_file[j-win_size+k].lower()])
                        else:
                            word_embed = np.add(word_embed, glove_model['unk'])  
                    
                    actual_tag = tag_in_file[j]
                    if actual_tag in actual_tag_count.keys():
                        actual_tag_count[actual_tag] = actual_tag_count[actual_tag] + 1
                    # Get the predicted tag
                    nearestCosineDist = 2
                    predicted_tag = ""
                    for tag in tagEmbedding:
                        dist = 1 - (dot(tagEmbedding[tag], word_embed)/(norm(tagEmbedding[tag]) * norm(word_embed)))
                        if dist < nearestCosineDist:
                            predicted_tag = tag
                            nearestCosineDist = dist
                    if actual_tag == predicted_tag:
                        if actual_tag in matched_tag_count.keys():
                            matched_tag_count[actual_tag] = matched_tag_count[actual_tag] + 1                       
                        

In [71]:
print('======= Accuracy Class Wise =====================\n')

print('Class    Matched Total %')
print('--------------------------')
print ('PER_Others: '+str(matched_tag_count['PER_Others'])+ ' ' + str(actual_tag_count['PER_Others'])+ ' ' + str(matched_tag_count['PER_Others']*100/actual_tag_count['PER_Others'])+'%')
print ('PER_Victim: '+str(matched_tag_count['PER_Victim'])+ ' ' + str(actual_tag_count['PER_Victim'])+ ' '+str(matched_tag_count['PER_Victim']*100/actual_tag_count['PER_Victim'])+'%')
print ('PER_Accused: '+str(matched_tag_count['PER_Accused'])+ ' ' + str(actual_tag_count['PER_Accused'])+ ' '+str(matched_tag_count['PER_Accused']*100/actual_tag_count['PER_Accused'])+'%')
print ('ORG_Victim: '+str(matched_tag_count['ORG_Victim']) + ' '+ str(actual_tag_count['ORG_Victim'])+ ' '+str(matched_tag_count['ORG_Victim']*100/actual_tag_count['ORG_Victim'])+'%')
print ('ORG_Accused: '+str(matched_tag_count['ORG_Accused']) + ' '+ str(actual_tag_count['ORG_Accused'])+ ' '+str(matched_tag_count['ORG_Accused']*100/actual_tag_count['ORG_Accused'])+'%')
print ('ORG_Others: '+str(matched_tag_count['ORG_Others']) + ' '+ str(actual_tag_count['ORG_Others'])+ ' '+str(matched_tag_count['ORG_Others']*100/actual_tag_count['ORG_Others'])+'%')
print ('LOC_Accused: '+str(matched_tag_count['LOC_Accused'])+ ' ' + str(actual_tag_count['LOC_Accused'])+ ' '+str(matched_tag_count['LOC_Accused']*100/actual_tag_count['LOC_Accused'])+'%')
print ('LOC_Others: '+str(matched_tag_count['LOC_Others'])+ ' ' + str(actual_tag_count['LOC_Others'])+ ' '+str(matched_tag_count['LOC_Others']*100/actual_tag_count['LOC_Others'])+'%')
print ('LOC_Event: '+str(matched_tag_count['LOC_Event']) + ' '+ str(actual_tag_count['LOC_Event'])+ ' '+str(matched_tag_count['LOC_Event']*100/actual_tag_count['LOC_Event'])+'%')
print ('LOC_Victim: '+str(matched_tag_count['LOC_Victim']) + ' '+ str(actual_tag_count['LOC_Victim'])+ ' '+str(matched_tag_count['LOC_Victim']*100/actual_tag_count['LOC_Victim'])+'%')

======= Accuracy Class Wise =====================

Class    Matched Total %
--------------------------
PER_Others: 1175 2191 53%
PER_Victim: 97 251 38%
PER_Accused: 385 1029 37%
ORG_Victim: 15 82 18%
ORG_Accused: 317 730 43%
ORG_Others: 907 2911 31%
LOC_Accused: 0 141 0%
LOC_Others: 689 2072 33%
LOC_Event: 784 1809 43%
LOC_Victim: 9 37 24%
